In [ ]:
#Imports all the needed libraries
import kagglehub
import pandas as pd
import os
import torch
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel
import numpy as np


print("All is good")



In [ ]:
# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
# Download the dataset from Kagglehub
path = kagglehub.dataset_download("aaron7sun/stocknews")

# Find the CSV file inside the downloaded folder
csv_file_one = None
for file in os.listdir(path):
    if file.endswith(".csv") and "Combined" in file:
        csv_file_one = os.path.join(path, file) #Sets the variable csv_file_one as the path
        break

# Checks if the right csv data file is found
if csv_file_one != None:
    # The following code utilizes pandas to load the dataset and prints out the first 5 lines
    df1 = pd.read_csv(csv_file_one)
    print("Sucess!!!!!!")
    print(df1.head())
else:
    print("No CSV file found in the downloaded dataset.")



In [ ]:
#Changes the column named Label to a new name Price Increased
df1 = df1.rename(columns={'Label': 'Price Increased'})


In [ ]:
#Goes through the dataset and converts 1s and 0s into boolean values
for i in range(len(df1)):
    if df1.at[i, 'Price Increased'] == 1:
        df1.at[i, 'Price Increased'] = True
    elif df1.at[i, 'Price Increased'] == 0:
        df1.at[i, 'Price Increased'] = False


In [ ]:
#This function converts a string into a vector which allows the ml model to understand the article title
def getVectorFromString(text):
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=64, return_tensors="pt")

    with torch.no_grad():
        output = bert_model(**tokens)

    # Takes the mean to get one single vector
    embedding = output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

NameError: name 'df1' is not defined

In [ ]:
#Displays rows that have a N/A value
df1 = df1.fillna("")
nan_rows = df1[df1.isna().any(axis=1)]
print(nan_rows)

<ipython-input-5-44605c23c328>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Price Increased'] = df1['Price Increased'].replace({1: True, 0: False})


,Date,Price Increased,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,False,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,True,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,False,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,False,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,True,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to 

In [ ]:
#List of all headline column names
headline_columns = []
for i in range(1, 26):
    column_name = 'Top' + str(i)
    headline_columns.append(column_name)

#This function takes the average of all the vectors in a row.
def averageOfAllVectors(row):
    embeddings = []

    for col in headline_columns:
        text = row[col]
        vector = getVectorFromString(text)
        embeddings.append(vector)

    #Computes the average vector of all the article title vectors
    average_vector = np.mean(embeddings, axis=0)
    return average_vector

#Creates a new column that takes the ve tor average for each row
df1['embedding'] = df1.apply(averageOfAllVectors, axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Sets the input and target variables for the model
X = pd.DataFrame(df1['embedding'].to_list())
y = df1['Price Increased']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the target into integers rather than boolean
y_encoded = y.astype(int)



In [ ]:
from sklearn.model_selection import train_test_split

#Splits data into 80% for training and 20% for testing to evaluate the accuracy
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=23)



In [ ]:
from sklearn.linear_model import LogisticRegression

#Creates the model object and trains the model with the training data
model = LogisticRegression(solver='liblinear', max_iter=10000)

model.fit(X_train, y_train)



In [ ]:
#Creates predicitons based on the testing data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(round(accuracy, 2)))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt



#Generates a visualization of the models accuracy
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()